In [ ]:
!pip install geopandas
import geopandas as gpd
!pip install mapclassify
import matplotlib.pyplot as plt
from mapclassify import classify
import pandas as pd
import numpy as np

# **Leyendo y escribiendo archivos**

**GeoPandas** puede leer casi cualquier formato de datos espaciales basado en vectores, incluyendo ESRI shapefile, archivos GeoJSON y más utilizando el comando: `geopandas.read_file()`

el cual devuelve un objeto `GeoDataFrame`. Esto es posible porque geopandas hace uso de la librería **fiona**

Hay distintas formas de leer los archivos:

* ZIP files:

`zipfile = "zip:///Users/name/Downloads/cb_2017_us_state_500k.zip"`

`states = geopandas.read_file(zipfile)`

Si el dataset se encuentra en una carpeta del archivo ZIP, deberá añadir su nombre:

`zipfile = "zip:///Users/name/Downloads/gadm36_AFG_shp.zip!data"`




* Files desde URL:

`url = "http://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson"`

`df = geopandas.read_file(url)`

* O bien, importar files desde tu equipo local

En este caso vamos a utilizar archivos desde una URL

In [ ]:
estados = gpd.read_file(r"https://raw.githubusercontent.com/BobadillaE/Pobreza_Mexico_RStudio/main/Datasets%20empleados/formatted4.json")

In [ ]:
municipios = gpd.read_file(r"https://raw.githubusercontent.com/angelnmara/geojson/master/MunicipiosMexico.json")

Para escribir un `GeoDataFrame` en un archivo con distinto formato se usa: `GeoDataFrame.to_file()`. 

El formato de archivo por defecto es Shapefile, pero puede especificar el suyo propio con la keyword en el `driver`:

`gdf.to_file("my_file.geojson", driver="GeoJSON")`



---



# **`Pandas` dataframes y `GeoPandas` Geodataframes**

Podemos tener dos dataframes distintos, un `pandas.DataFrame` y un `geopandas.GeoDataFrame`, y aplicar todas las funciones de pandas para juntar dos dataframes, lo que resultaría en este caso en un `geopandas.GeoDataFrame`.

In [ ]:
pd_df = pd.read_csv(r"https://raw.githubusercontent.com/BobadillaE/Pobreza_Mexico_RStudio/main/Datasets%20empleados/DF_MX.xlsx%20-%20Hoja1.csv")

In [ ]:
pd_df.head()

In [ ]:
merged = pd.merge(estados, pd_df, on='id')

In [ ]:
merged.head()

# **Algunos conceptos básicos**

## Áreas

Para medir el `area` de cada polígono se accede al atributo `GeoDataFrame.area`, que devuelve un `pandas.Series`. Esto nos devolverá una nueva columna con el área correspondiente de cada polígono

In [ ]:
estados["area"] = estados.area
municipios["area"]= municipios.area

In [ ]:
estados.head()

## Límite del polígono y centroides

Para obtener el límite (perímetro) de cada polígono (`LineString`), acceda a `GeoDataFrame.boundary`. En este caso no tendremos una columna con puros números, sino una columna nueva de geometría

In [ ]:
estados['boundary'] = estados.boundary
municipios['boundary'] = municipios.boundary

In [ ]:
estados.head()

También podemos crear nuevas geometrías, que podrían ser, por ejemplo su centroide, el cual nos devuelve igualmente una nueva columna de geometría

In [ ]:
estados['centroid'] = estados.centroid
municipios['centroid'] = municipios.centroid

In [ ]:
estados.head()

## Medir distancias

También podemos medir a qué distancia se encuentra cada centroide de la ubicación de un primer centroide

In [ ]:
first_point = estados['centroid'].iloc[23]
estados['distance'] = estados['centroid'].distance(first_point)

In [ ]:
estados['distance']

In [ ]:
estados.head()

Como `geopandas.GeoDataFrame` es una subclase de `pandas.DataFrame`, disponemos de toda la funcionalidad de pandas para utilizarla, por ejemplo, para calcular la media de las distancias medidas anteriormente, acceda a la columna 'distance' y llame al método mean() sobre ella

In [ ]:
estados['distance'].mean()



---



# ***Indexing* y *Selecting* de datos**

Además de los métodos estándar de `pandas`, `GeoPandas` también proporciona indexación/selección basada en coordenadas con el indexador `cx`, que corta nuestro mapa utilizando una caja delimitadora. Las geometrías en el GeoSeries o GeoDataFrame que intersecan la caja delimitadora serán devueltas.

In [ ]:
estados_sur = estados.cx[:, :18]

estados_sur.plot(figsize=(10, 3))

In [ ]:
municipios_norte = municipios.cx[:, 25:]

municipios_norte.plot(figsize=(10, 3))



---



# **Haciendo mapas y usando algunas herramientas**

## Mapas Básicos

In [ ]:
estados.plot()

In [ ]:
municipios.boundary.plot()

In [ ]:
merged.centroid.plot()

## Choropleth Maps

`GeoPandas` facilita la creación de mapas *Choropleth* (mapas en los que el color de cada forma se basa en el valor de una variable asociada).

Sólo habría que utilizar el comando `plot` con el argumento `column` ajustado a la columna cuyos valores desea utilizar para asignar colores

In [ ]:
merged.plot(column='pibPC', legend=True)

Se puede modificar el label con `matplotlib`

In [ ]:
fig, ax = plt.subplots(1, 1)
merged.plot(column='pibPC',
           ax=ax,
           legend=True,
           legend_kwds={'label': "PIB per Capita de los estados Mexicanos",
                        'orientation': "horizontal"})

O incluso el color

In [ ]:
fig, ax = plt.subplots(1, 1)
merged.plot(column='pibPC',
           ax=ax,
           legend=True,
            cmap= 'coolwarm',
           legend_kwds={'label': "PIB per Capita de los estados Mexicanos",
                        'orientation': "horizontal"})

La opción de `scheme` puede establecerse en cualquier esquema proporcionado por `mapclassify` (p. ej. 'box_plot', 'equal_interval', 'fisher_jenks', 'fisher_jenks_sampled', 'headtail_breaks', 'jenks_caspall', 'jenks_caspall_forced', 'jenks_caspall_sampled', 'max_p_classifier', 'maximum_breaks', 'natural_breaks', 'quantiles', 'percentiles', 'std_mean' o 'user_defined').

In [ ]:
merged.plot(column='pibPC', cmap='OrRd', scheme='quantiles', legend=True)

## Mapas con distintas *layers*

### Proyecciones

Antes de aplicar distintas *layers* de poligonos tenemos que asegurarnos que se encuentren en el mismo *CRS* (*Coordinate Reference Systems*)

El CRS es importante porque las formas geométricas en un objeto GeoSeries o GeoDataFrame son simplemente una colección de coordenadas en un espacio arbitrario. Un CRS le dice a Python cómo se relacionan esas coordenadas con lugares de la Tierra

Como referencia, algunas proyecciones muy comunes y sus códigos EPSG:

* Latitud/Longitud WGS84: "EPSG:4326"

* Zonas UTM (Norte): "EPSG:32633"

* Zonas UTM (Sur): "EPSG:32733"

Para revisar nuestro CRS simplemente usamos el comando `GeoDataFrame.crs`

In [ ]:
estados.crs

Para cambiar nuestro CRS

In [ ]:
municipios = municipios.to_crs(estados.crs)


Una vez que tenemos nuestros CRS en orden, tenemos dos formas de hacer `plots` con distintas `layers` de polígonos, lineas o centroides

**Método 1**

In [ ]:
base = estados.plot(color='white', edgecolor='black')

municipios.centroid.plot(ax=base, marker='*', color='red', markersize=5)

**Método 2: usando objetos de `matplotlib`**

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
estados.plot(ax=ax, color='white', edgecolor='black')
municipios.centroid.plot(ax=ax, marker='*', color='red', markersize=5)
plt.show()

### Controlar el orden de varias *layers* en un *plot*

Al trazar varias *layers*, utilice `zorder` para controlar el orden de las *layers* trazadas. Cuanto más bajo sea el `zorder`, más abajo estará la capa en el mapa y viceversa.

In [ ]:
mdf1.plot(ax=municipios.centroid.plot(color='red',markersize=0.1), color='white', edgecolor='black')

In [ ]:
mdf1.plot(ax=municipios.centroid.plot(color='red',markersize=0.1, zorder=2), color='white', edgecolor='black', zorder=1);

## Datos faltantes

En algunos casos, es posible que se desee hacer *plots* con datos que contengan *missing values*, ya que para algunas características simplemente se desconoce el valor. Esto es posible con la librería de `numpy`

In [ ]:
merged_nan = pd.merge(estados, pd_df, on='id')
merged_nan.loc[np.random.choice(merged_nan.index, 30), 'pibPC'] = np.nan
merged_nan.plot(column='pibPC')

In [ ]:
merged_nan.plot(
    column="pibPC",
    cmap= 'OrRd',
    legend=True,
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
        "hatch": "///",
        "label": "Missing values",
    },
)

## *Plots* de Pandas



Los métodos de plotting también permiten diferentes estilos de ploteo desde `pandas` junto con el ploteo `geo` por defecto. Se puede acceder a estos métodos utilizando el argumento kind keyword en plot(), e incluyen

* `geo` para trazado de mapas

* `line` para gráficos de líneas

* `bar` o barh para gráficos de barras

* `hist` para histograma

* `box` para boxplot

* `kde` o `density` para gráficos de densidad

* `area` para gráficos de área

* `scatter` para gráficos de dispersión

* `hexbin` para gráficos hexagonales

* `pie` para gráficos circulares

In [ ]:
merged.plot(kind='scatter', x="pibPC", y="pob_ind")



---



# **Mapas interactivos**

Además de los gráficos estáticos, `geopandas` puede crear mapas interactivos basados en la biblioteca `folium`

La opción más sencilla es utilizar `GeoDataFrame.explore()`:

In [ ]:
estados.explore()

In [ ]:
municipios.centroid.explore()

In [ ]:
merged.boundary.explore()

Un poco más complejo

In [ ]:
estados.explore(
     column="name", # make choropleth based on "pibPC" column
     tooltip="name", # show "pibPC" value in tooltip (on hover)
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Set1", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black") # use black outline
    )

In [ ]:
import folium


m = merged.explore(
     column="pibPC", 
     cmap='OrRd',
     scheme='quantiles',
     legend=True, 
     k=10, 
     legend_kwds=dict(colorbar=False), 
     name="Estados"
)

municipios.centroid.explore(
     m=m, # pass the map object
     color="k", 
     marker_kwds=dict(radius=2, fill=True), 
     name="Municipios" 
)

folium.TileLayer('Stamen Toner', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m 



---



# **GeoCoding**

`GeoPandas` admite la geocodificación (es decir, la conversión de nombres de lugares a ubicaciones en la Tierra) a través de `geopy`, una dependencia opcional de geopandas

El siguiente ejemplo muestra cómo obtener las ubicaciones de los estados de México y traza esas ubicaciones junto con el archivo detallado de los límites de los estados

In [ ]:
estados_geocoding = gpd.tools.geocode(estados.name)

In [ ]:
estados_geocoding

In [ ]:
fig, ax = plt.subplots()

estados.plot(ax=ax, color="white", edgecolor="black");

estados_geocoding.plot(ax=ax, color="red")